# DOC Correction

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd

Found https://www.mass.gov/service-details/data-dashboards which has a ton of information. Not sure how to get df from  Tableau interactive though. Can't find where cases originated from either in that data. Maybe what we can do it take number of people Suffolk DA sent to jail out of total admissions? Ask Bobby if we can get that data on SCDAO side

I also got to here https://www.mass.gov/report/research-reports <br>
https://www.mass.gov/lists/communities-inmates-released-to has the communities inmates are released to <br>
https://www.mass.gov/lists/prison-population-trends also has communities inmates are released for the years we are looking at <br>
https://www.mass.gov/lists/admissions-and-releases has Criminally Sentenced New Court Commitments by Court Jurisdiction which is the closest to what we want

The 2019 third quarter document has a total of 

In [5]:
437+433+359+384

1613

New admissions for calendar year 2018.<br> 
Asterisk at bottom of chart tells us: Excludes county, federal, and out-of-state inmates. We don't have to worry ab federal inmates. I don't know how many out of state inmates there are. In the 2016 report on where inmates are released to, 131 out of 2328 inmates were released to outside MA which is trivial, so we can ignore. According to Bobby, "county inmates" are inmates the DOC is holding instead of local sheriff's departments because of concerns about fighting in local pens.

My current understanding: there are a group of total admissions to DOC custody each year. Of these, some are pre-trial, some are criminal, some are civil, and some are pre-trial. <br>
Question for Bobby: do we care about the civil admissions to DOC custody? Answer is no<br>



Ok so current plan: all non-civil admissions we count. Out of this group of non-civil admissions, we assume that % of admissions that are from suffolk is the same as the % of new court commitments that are from suffolk. It's the best we can do with the information on mass.gov

We only need two pdf's to span 2016 through (most of) 2019 where the data ends. They are the admissions releases report https://www.mass.gov/lists/admissions-and-releases from 2017 quarter one and 2019 quarter 3. Both are saved in /Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/data/Agency_Corrections/DOC/Admissions_Releases...

In [6]:
from itertools import permutations, combinations, product
  
 
quarters =[str(x[0]) + " " + x[1] for x in  product(list(range(2016,2020)), ["Q" + str(x) for x in range(1,5)]) ]

In [7]:
df = pd.DataFrame(columns = quarters)

df.loc["Suffolk Admits", "2016 Q1"] = 72
df.loc["Total Admits", "2016 Q1"] = 439

df.loc["Suffolk Admits", "2016 Q2"] = 79
df.loc["Total Admits", "2016 Q2"] = 416

df.loc["Suffolk Admits","2016 Q3"] = 41 
df.loc["Total Admits","2016 Q3"] = 385

df.loc["Suffolk Admits","2016 Q4"] = 74
df.loc["Total Admits","2016 Q4"] = 387

df.loc["Suffolk Admits","2017 Q1"] = 71
df.loc["Total Admits","2017 Q1"] = 500

df.loc["Suffolk Admits","2017 Q2"] = 83
df.loc["Total Admits","2017 Q2"] = 440

df.loc["Suffolk Admits","2017 Q3"] = 61
df.loc["Total Admits","2017 Q3"] = 372

df.loc["Suffolk Admits","2017 Q4"] = 77
df.loc["Total Admits","2017 Q4"] = 426

df.loc["Suffolk Admits","2018 Q1"] = 83
df.loc["Total Admits","2018 Q1"] = 437

df.loc["Suffolk Admits","2018 Q2"] = 84
df.loc["Total Admits","2018 Q2"] = 433

df.loc["Suffolk Admits","2018 Q3"] = 72
df.loc["Total Admits","2018 Q3"] = 359

df.loc["Suffolk Admits","2018 Q4"] = 76
df.loc["Total Admits","2018 Q4"] = 384

df.loc[:,df.columns.str.contains("2018")].iloc[0,:].sum()/\
df.loc[:,df.columns.str.contains("2018")].iloc[1,:].sum()

0.19528828270303783

In [8]:
df.loc["Suffolk Admits","2019 Q1"] = 70
df.loc["Total Admits","2019 Q1"] = 431

df.loc["Suffolk Admits","2019 Q2"] = 90
df.loc["Total Admits","2019 Q2"] = 433

df.loc["Suffolk Admits","2019 Q3"] = 64
df.loc["Total Admits","2019 Q3"] = 364

In [9]:
df

,2016 Q1,2016 Q2,2016 Q3,2016 Q4,2017 Q1,2017 Q2,2017 Q3,2017 Q4,2018 Q1,2018 Q2,2018 Q3,2018 Q4,2019 Q1,2019 Q2,2019 Q3,2019 Q4
Suffolk Admits,72,79,41,74,71,83,61,77,83,84,72,76,70,90,64,NaN
Total Admits,439,416,385,387,500,440,372,426,437,433,359,384,431,433,364,NaN


In [10]:
correction_dict = {}
for y in list(range(2016, 2020)):
    little = df.loc[:, df.columns.str.contains(str(y))]
    s = little.sum(axis=1)
    correction_dict[y] = s[0]/s[1]
    

In [11]:
correction_dict

{2016: 0.16349108789182545,
 2017: 0.16800920598388952,
 2018: 0.19528828270303783,
 2019: 0.18241042345276873}

Ok great. For the next step we could take the average quartley population from figure 1.2 in the admissions and releases report I use and say that we are only looking for budget used to house non-civil commitments, which we estimate as % of population that is non-civil. I think that makes sense, DA's office doesn't send people to civil lock-ups

In [22]:
def get_criminal_pop(pop_df):
    out_dict = pd.Series()
    for y in list(range(2016,2020)):
        little = pop_df.loc[:,pop_df.columns.str.contains(str(y))]
        s = little.sum(axis=1)
        out_dict.loc[y] = 1 - s[0]/s[1]
    return out_dict

In [23]:
pop_df = pd.DataFrame(columns = quarters)


pop_df.loc["Civil Pop", "2016 Q1"] = 607
pop_df.loc["Total Pop", "2016 Q1"] = 10027

pop_df.loc["Civil Pop", "2016 Q2"] = 604
pop_df.loc["Total Pop", "2016 Q2"] = 9872

pop_df.loc["Civil Pop", "2016 Q3"] = 630
pop_df.loc["Total Pop", "2016 Q3"] = 9789

pop_df.loc["Civil Pop", "2016 Q4"] = 581
pop_df.loc["Total Pop", "2016 Q4"] = 9596

pop_df.loc["Civil Pop", "2017 Q1"] = 572
pop_df.loc["Total Pop", "2017 Q1"] = 9527

pop_df.loc["Civil Pop", "2017 Q2"] = 578 
pop_df.loc["Total Pop", "2017 Q2"] = 9468

pop_df.loc["Civil Pop", "2017 Q3"] = 594 
pop_df.loc["Total Pop", "2017 Q3"] = 9454

pop_df.loc["Civil Pop", "2017 Q4"] = 562
pop_df.loc["Total Pop", "2017 Q4"] = 9310 

pop_df.loc["Civil Pop", "2018 Q1"] = 543
pop_df.loc["Total Pop", "2018 Q1"] = 9167

pop_df.loc["Civil Pop", "2018 Q2"] = 571
pop_df.loc["Total Pop", "2018 Q2"] = 9125 

pop_df.loc["Civil Pop", "2018 Q3"] = 551
pop_df.loc["Total Pop", "2018 Q3"] = 9032 

pop_df.loc["Civil Pop", "2018 Q4"] = 489
pop_df.loc["Total Pop", "2018 Q4"] = 8855 

pop_df.loc["Civil Pop", "2019 Q1"] = 499
pop_df.loc["Total Pop", "2019 Q1"] = 8807

pop_df.loc["Civil Pop", "2019 Q2"] = 567
pop_df.loc["Total Pop", "2019 Q2"] = 8799

pop_df.loc["Civil Pop", "2019 Q3"] = 644
pop_df.loc["Total Pop", "2019 Q3"] = 8754 

In [24]:
pcnt_criminal = get_criminal_pop(pop_df)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [25]:
pcnt_criminal

2016    0.938346
2017    0.938928
2018    0.940463
2019    0.935129
dtype: float64

In [26]:
df = pd.DataFrame(columns = list(range(2016,2020)))

In [29]:
df.loc["index 1"] = [1,2,3,4]
df.loc["index 2"] = [5,6,7,8]

In [31]:
df*pcnt_criminal

,2016,2017,2018,2019
index 1,0.938346,1.87786,2.82139,3.74052
index 2,4.69173,5.63357,6.58324,7.48103


Cool, that seems good. 